### Vectorizing tabular fields

This notebook covers simple methods to vectorize tabular data using the same dataset as in the other examples, but this time ignoring the text of the question.

In [3]:
import pandas as pd
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')


In [4]:
data_path = Path("../data/posts.csv")
df = pd.read_csv(data_path)

In [5]:
def get_normalized_series(df, col):
    """
    Get a normalized version of a column
    :param df: DataFrame
    :param col: column name
    :return: normalized series using Z-score
    """
    return (df[col] - df[col].mean()) / df[col].std()

t's pretend we wanted to predict the score from the tags, number of comments, and question creation date. Here is what the data looks like

In [6]:
df["is_question"] = df["PostTypeId"] == 1

tabular_df = df[df["is_question"]][["Tags", "CommentCount", "CreationDate", "Score"]]
tabular_df.head()

,Tags,CommentCount,CreationDate,Score
0,<resources><first-time-author>,7,2010-11-18T20:40:32.857,32
1,<fiction><grammatical-person><third-person>,0,2010-11-18T20:42:31.513,23
2,<publishing><novel><agent>,1,2010-11-18T20:43:28.903,36
3,<plot><short-story><planning><brainstorming>,1,2010-11-18T20:43:59.693,34
4,<fiction><genre><categories>,1,2010-11-18T20:45:44.067,22


In order to use this data as input to a model, we need to give it a suitable numerical representation. To do so, we will do three things here:

1. Normalize numerical input features to limit the impact of outliers

2. Transform the date feature in a way that makes it easier to understand for a model.

3. Get dummy variables from categorical features so a model can ingest them.

First, we normalize the data to reduce the effect of outliers on downstream model performance.

In [7]:
tabular_df["NormComment"]= get_normalized_series(tabular_df, "CommentCount")
tabular_df["NormScore"]= get_normalized_series(tabular_df, "Score")

In [9]:
tabular_df.head()

,Tags,CommentCount,CreationDate,Score,NormComment,NormScore
0,<resources><first-time-author>,7,2010-11-18T20:40:32.857,32,1.326954,3.383644
1,<fiction><grammatical-person><third-person>,0,2010-11-18T20:42:31.513,23,-0.883784,2.246533
2,<publishing><novel><agent>,1,2010-11-18T20:43:28.903,36,-0.567964,3.889026
3,<plot><short-story><planning><brainstorming>,1,2010-11-18T20:43:59.693,34,-0.567964,3.636335
4,<fiction><genre><categories>,1,2010-11-18T20:45:44.067,22,-0.567964,2.120187


Now, let's represent dates in a way that would make it easier for a model to extract patterns.

In [10]:
# Convert our date to a pandas datetime
tabular_df["date"] = pd.to_datetime(tabular_df["CreationDate"])

# Extract meaningful features from the datetime object
tabular_df["year"] = tabular_df["date"].dt.year
tabular_df["month"] = tabular_df["date"].dt.month
tabular_df["day"] = tabular_df["date"].dt.day
tabular_df["hour"] = tabular_df["date"].dt.hour

In [11]:
tabular_df.head()

,Tags,CommentCount,CreationDate,Score,NormComment,NormScore,date,year,month,day,hour
0,<resources><first-time-author>,7,2010-11-18T20:40:32.857,32,1.326954,3.383644,2010-11-18 20:40:32.857,2010,11,18,20
1,<fiction><grammatical-person><third-person>,0,2010-11-18T20:42:31.513,23,-0.883784,2.246533,2010-11-18 20:42:31.513,2010,11,18,20
2,<publishing><novel><agent>,1,2010-11-18T20:43:28.903,36,-0.567964,3.889026,2010-11-18 20:43:28.903,2010,11,18,20
3,<plot><short-story><planning><brainstorming>,1,2010-11-18T20:43:59.693,34,-0.567964,3.636335,2010-11-18 20:43:59.693,2010,11,18,20
4,<fiction><genre><categories>,1,2010-11-18T20:45:44.067,22,-0.567964,2.120187,2010-11-18 20:45:44.067,2010,11,18,20


And finally let's transform tags into dummy variables using pandas' [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) function, with each tag being assigned an index that will take the value "1" only if it is present in the given row.

In [12]:
# Select our tags, represented as strings, and transform them into arrays of tags
tags = tabular_df["Tags"]
clean_tags = tags.str.split("><").apply(
    lambda x: [a.strip("<").strip(">") for a in x])

In [13]:
clean_tags

0                      [resources, first-time-author]
1         [fiction, grammatical-person, third-person]
2                          [publishing, novel, agent]
3        [plot, short-story, planning, brainstorming]
4                        [fiction, genre, categories]
                             ...                     
44815           [word-choice, poetry, lyrics, rhythm]
44822                                   [short-story]
44823                           [criticism, metaphor]
44825             [creative-writing, science-fiction]
44826    [plot, tools, collaboration, writing-groups]
Name: Tags, Length: 10495, dtype: object

In [21]:
# Use pandas' get_dummies to get dummy values 
tag_columns = pd.get_dummies(clean_tags.apply(pd.Series).stack()).sum(level=0)
all_tags = tag_columns.astype(bool).sum(axis=0).sort_values(ascending=False)

In [22]:
# select only tags that appear over 500 times
top_tags = all_tags[all_tags > 500]
top_tag_columns = tag_columns[top_tags.index]

In [23]:

top_tag_columns.head()

,creative-writing,fiction,style,characters,technique,novel,publishing,character-development,plot
0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,1,1,0,0
3,0,0,0,0,0,0,0,0,1
4,0,1,0,0,0,0,0,0,0


In [24]:
# Add our tags back into our initial DataFrame
final = pd.concat([tabular_df, top_tag_columns], axis=1)

# Keeping only the vectorized features
col_to_keep = ["year", "month", "day", "hour", "NormComment",
               "NormScore"] + list(top_tags.index)
final_features = final[col_to_keep]

In [26]:
final_features.head()

,year,month,day,hour,NormComment,NormScore,creative-writing,fiction,style,characters,technique,novel,publishing,character-development,plot
0,2010,11,18,20,1.326954,3.383644,0,0,0,0,0,0,0,0,0
1,2010,11,18,20,-0.883784,2.246533,0,1,0,0,0,0,0,0,0
2,2010,11,18,20,-0.567964,3.889026,0,0,0,0,0,1,1,0,0
3,2010,11,18,20,-0.567964,3.636335,0,0,0,0,0,0,0,0,1
4,2010,11,18,20,-0.567964,2.120187,0,1,0,0,0,0,0,0,0


Voila! Our tabular data is now ready to be used for a model.